# Lab 4: Functional Programming
Look at you go! Congratulations on making it to the second part of the lab!

These assignments are *absolutely not required*! Even if you're here, you may find it more valuable to skim the problems here and attempt the problems that are most interesting to you - and that's perfectly fine. Don't feel any need to complete them in sequential order at this point.

(Though we'd recommend the decorators problem, as a fun challenge!)

### Linear Algebra (Challenge)

These challenge problems test your ability to write compact Python functions using the tools of functional programming and some good old-fashioned cleverness. As always, these challenge problems are optional, and are much harder than the rest of the lab. These challenge problems also focus heavily on linear algebra, so if you are less familiar with linear algebra concepts, we recommend that you skip over this portion.

Also, Python has incredible library support for working with these mathematical concepts through a package named `numpy`, so we will almost never write linear algebra code from scratch.

#### Dot Product
Write a one-liner in Python that takes the dot product of two lists `u` and `v`. You can assume that the lists are the same size, and are standard Python lists (not anything special, like `numpy.ndarray`s). For example, `dot_product([1, 3, 5], [2, 4, 6])` should return `44` (since `1 * 2 + 3 * 4 + 5 * 6 = 44`).

In [ ]:
def dot_product(u, v):
    """Return the dot product of two equal-length lists of numbers."""
    pass

#### Matrix Transposition
Write a one-liner in Python to transpose a matrix. Assume that the input matrix is a tuple-of-tuples that represents a valid matrix, not necessarily square. Again, do not use `numpy` or any other libraries - just raw data structure manipulation and our functional tools.

Not only can you do this in one line - you can even do it in 14 characters!

For example,

```Python
matrix = (
    (1, 2, 3, 4),
    (5, 6, 7, 8),
    (9,10,11,12)
)

transpose(matrix)
# returns 
# (
#     (1, 5, 9),
#     (2, 6, 10),
#     (3, 7, 11),
#     (4, 8, 12)
# )
```


In [ ]:
def transpose(m):
    """Return the transpose of a matrix represented as a rectangular tuple-of-tuples."""
    pass

#### Matrix Multiplication
Write another one-liner in Python to take the product of two matrices `m1` and `m2`. You can use the `dot_product` and `transpose` functions you already wrote.

In [ ]:
def matmul(m1, m2):
    """Return the matrix multiplication of two matrices as rectangular 2D tuples."""
    pass

#### Lazy Generation
Rewrite your `transpose` and `matmul` functions above so that they are lazily evaluated. That is, rows (or columns) of the output matrix shouldn't be computed when the function is called.

## Building Decorators

Recall that a decorator is a special type of function that accepts a function as an argument and returns a new function which (usually) wraps some of the behavior of the supplied function.

Furthermore, recall that the `@decorator` syntax is syntactic sugar.

```Python
@decorator
def fn():
    pass
```

is equivalent to

```Python
def fn():
    pass
fn = decorator(fn)
```


### Review

In lecture, we implemented the `debug` decorator.

```Python
def debug(function):
    def wrapper(*args, **kwargs):
        print("Arguments:", args, kwargs)
        return function(*args, **kwargs)
    return wrapper
```

Take a moment, with a partner, and make sure you understand what is happening in the above lines. Why are the arguments to wrapper on the second line `*args` and `**kwargs` instead of something else? What would happen if we didn't `return wrapper` at the end of the function body?

### Automatic Caching
Write a decorator `cache` that will automatically cache any calls to the decorated function. You can assume that all arguments passed to the decorated function will always be hashable types.

```Python
def cache(function):
    pass  # Your implementation here
```

In pseudocode, to accomplish this you will
```
take in some function f
build a new function g: when called with some arguments...
    if we have seen these arguments before:
        return a saved result for these arguments
    otherwise:
        compute and return result of calling f with these arguments and save the result in some data structure
return g
```

For example, you should be able to use this decorator as follows:

```Python
@cache
def fib(n):
    return fib(n-1) + fib(n-2) if n > 2 else 1

fib(10)  # 55 (takes a moment to execute)
fib(10)  # 55 (returns immediately)
fib(100) # doesn't take forever
fib(400) # doesn't raise RuntimeError
```

Hint: You can set arbitrary attributes on a function (e.g. `fn._cache`). When you do so, the attribute-value pair also gets inserted into `fn.__dict__`. Take a look for yourself. Are the extra attributes and `.__dict__` always in sync?

#### Cache Options (Challenge)

Add `maxsize` and `eviction_policy` keyword arguments, with reasonable defaults (perhaps `maxsize=None` as a sentinel), to your `cache` decorator. `eviction_policy` should be one of `'LRU'`, `'MRU'`, or `'random'`. It can be tricky to figure out how to construct a decorator with arguments.

Also, add function attributes called `.cache_info` and `.cache_clear` which can be called to get aggregate statistics about the cache  and clear the cache, respectively.

#### Note
This caching decorator (with arguments!) is actually implemented as part of the language in `functools.lru_cache`

### Better Debugging Decorator
The `debug` decorator we wrote in class isn't very good. It doesn't tell us which function is being called, and it just dumps a tuple of positional arguments and a dictionary of keyword arguments - it doesn't even know what the names of the positional arguments are! If the default arguments aren't overridden, it won't show us their value either.

Use function attributes to improve our `debug` decorator into a `print_args` decorator that is "as good as you can make it."

```Python
def print_args(function):
    def wrapper(*args, **kwargs):
        # (1) You could do something here
        retval = function(*args, **kwargs)
        # (2) You could also do something here
        return retval
    return wrapper
```

*Hint: Consider using the attributes `fn.__name__` and `fn.__code__`. You'll have to investigate these attributes, but I will say that the `fn.__code__` code object contains a number of useful attributes - for instance, `fn.__code__.co_varnames`. Check it out! More information on function attributes is available in the latter half of Lab 3.*

#### Note
There are a lot of subtleties to this function, since functions can be called in a number of different ways. How does your `print_args` handle keyword arguments or even keyword-only arguments? Variadic positional arguments? Variadic keyword arguments? For more customization, look at `fn.__defaults__`, `fn.__kwdefaults__`, as well as other attributes of `fn.__code__`.

### Dynamic Type Checker (challenge)

Functions in Python can be optionally annotated by semantically-useless but structurally-valuable type annotations. For example:

```Python
def foo(a: int, b: str) -> bool:
    return b[a] == 'X'

foo.__annotations__  # => {'a': int, 'b': str, 'return': bool}
```

Write a runtime type checker, implemented as a decorator, that enforces that the types of arguments and the return value are valid.

```Python
def enforce_types(function):
    pass  # Your implementation here
```

For example:

```Python
@enforce_types
def foo(a: int, b: str) -> bool:
    if a == -1:
        return 'Gotcha!'
    return b[a] == 'X'

foo(3, 'abcXde')  # => True
foo(2, 'python')  # => False
foo(1, 4)  # prints "Invalid argument type for b: expected str, received int
foo(-1, '')  # prints "Invalid return type: expected bool, received str
```

There are lots of nuances to this function. What happens if some annotations are missing? How are keyword arguments and variadic arguments handled? What happens if the expected type of a parameter is not a primitive type? Can you annotate a function to describe that a parameter should be a list of strings? A tuple of (str, bool) pairs? A dictionary mapping strings to lists of integers? Read more about [advanced type hints](https://docs.python.org/3/library/typing.html) from the documentation.

As you make progress, show your decorator to a member of the course staff. 

#### Bonus: Optional Severity Argument
*Warning! This extension is very hard*

Extend the `enforce_types` decorator to accept a keyword argument `severity` which modifies the extent of the enforcement. If `severity == 0`, disable type checking. If `severity == 1` (which is the default), just print a message if there are any type violations. If `severity == 2`, raise a TypeError if there are any type violations.

For example:

```Python
@enforce_types_challenge(severity=2)
def bar(a: list, b: str) -> int:
    return 0

@enforce_types_challenge()  # Why are there parentheses here?
def baz(a: bool, b: str) -> str:
    return ''
```

## Credit
Credit goes to a lot of websites, whose names I've unfortunately forgotten along the way. Credit to everyone!

> With <3 by @sredmond